In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
# Загрузка данных (пример)
# Предположим, что данные уже загружены в DataFrame df
df = pd.read_csv('weather_data.csv')

# Предобработка данных
def prepare_data(df, sequence_length=7, target_length=7):
    # Разделение на обучающую и тестовую выборки
    train_size = int(0.8 * len(df))
    train_data = df.iloc[:train_size]
    test_data = df.iloc[train_size:]
    
    # Нормализация данных
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_data.drop(columns=['time', 'station']))
    test_scaled = scaler.transform(test_data.drop(columns=['time', 'station']))
    
    # Создание последовательностей
    def create_sequences(data, sequence_length, target_length):
        X, y = [], []
        for i in range(len(data) - sequence_length - target_length + 1):
            X.append(data[i:i+sequence_length])
            y.append(data[i+sequence_length:i+sequence_length+target_length])
        return np.array(X), np.array(y)
    
    X_train, y_train = create_sequences(train_scaled, sequence_length, target_length)
    X_test, y_test = create_sequences(test_scaled, sequence_length, target_length)
    
    return X_train, y_train, X_test, y_test, scaler

# Подготовка данных
X_train, y_train, X_test, y_test, scaler = prepare_data(df)

# Преобразование в тензоры PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# Создание DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Определение модели
class WeatherLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, sequence_length):
        super(WeatherLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(out.size(0), -1)  # Выравнивание для полносвязного слоя
        out = self.fc(out)
        return out


In [ ]:
# Параметры модели
input_size = X_train.shape[2]  # Количество признаков
hidden_size = 64
num_layers = 2
output_size = y_train.shape[1] * y_train.shape[2]  # Количество прогнозируемых значений
sequence_length = X_train.shape[1]

# Инициализация модели, функции потерь и оптимизатора
model = WeatherLSTM(input_size, hidden_size, num_layers, output_size, sequence_length).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y.reshape(batch_y.size(0), -1))
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Оценка модели
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test.reshape(y_test.size(0), -1))
    print(f'Test Loss: {test_loss.item():.4f}')